## 使用重复元素的网络(VGG)
$AlexNet$在$LeNet$的基础上增加了$3$个卷积层。但$AlexNet$作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然$AlexNet$指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

本节介绍$VGG$，$VGG$提出了可以通过**重复使用简单的基础块**来构建深度模型的思路。

### VGG块
$VGG$块的组成规律是：连续使用数个相同的填充为$1$、窗口形状为$3\times 3$的卷积层后接上一个步幅为$2$、窗口形状为$2\times 2$的最大池化层。**卷积层保持输入的高和宽不变，而池化层则对其减半**。

对于给定的感受野（与输出有关的输入图片的局部大小），采用**堆积的小卷积核优于采用大的卷积核**，因为可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。例如，在$VGG$中，使用了$3$个$3\times 3$卷积核来代替$7\times 7$卷积核，使用了$2$个$3\times 3$卷积核来代替$5\times 5$卷积核，这样做的主要目的是在保证具有相同感知野的条件下，提升了网络的深度，在一定程度上提升了神经网络的效果。

In [1]:
import time
import torch
from torch import nn, optim
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

def vgg_block(num_convs, in_channels, out_channels):
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
    blk.append(nn.MaxPool2d(2))
    return nn.Sequential(*blk)

### VGG网络
与$AlexNet$和$LeNet$一样，$VGG$网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个$vgg\_block$，其超参数由变量$conv\_arch$定义。该变量指定了每个$VGG$块里卷积层个数和输入输出通道数。全连接模块则跟$AlexNet$中的一样。

现在我们构造一个$VGG$网络。它有$5$个卷积块，前$2$块使用单卷积层，而后$3$块使用双卷积层。第一块的输入输出通道分别是$1$（因为下面要使用的$Fashion-MNIST$数据的通道数为$1$）和$64$，之后每次对输出通道数翻倍，直到变为$512$。因为这个网络使用了$8$个卷积层和$3$个全连接层，所以经常被称为$VGG-11$。

In [2]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 经过5个vgg_block，宽高会减半五次，变成224/32=7
fc_features = 512*7*7
fc_hidden_units = 4096

In [3]:
def vgg(conv_arch, fc_features, fc_hidden_units=4096):
    net = nn.Sequential()
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        net.add_module("vgg_block_"+str(i+1), vgg_block(num_convs, in_channels,out_channels))
    # 全连接层
    net.add_module("fc", nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(fc_features, fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units, fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units, 10)
    ))
    return net

下面构造一个高和宽均为$224$的单通道数据样本来观察每一层的输出形状。

In [4]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224)
# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape', X.shape)

vgg_block_1 output shape torch.Size([1, 64, 112, 112])
vgg_block_2 output shape torch.Size([1, 128, 56, 56])
vgg_block_3 output shape torch.Size([1, 256, 28, 28])
vgg_block_4 output shape torch.Size([1, 512, 14, 14])
vgg_block_5 output shape torch.Size([1, 512, 7, 7])
fc output shape torch.Size([1, 10])


可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成$7$后传入全连接层。与此同时，输出通道数每次翻倍，直到变成$512$。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。$VGG$这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

### 获取数据和训练模型
因为$VGG-11$计算上比$AlexNet$更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在$Fashion-MNIST$数据集上进行训练。

In [5]:
ratio = 8
small_conv_arch = [
    (1, 1, 64//ratio),
    (1, 64//ratio, 128//ratio),
    (2, 128//ratio, 256//ratio),
    (2, 256//ratio, 512//ratio),
    (2, 512//ratio, 512//ratio)
]
net = vgg(small_conv_arch, fc_features//ratio, fc_hidden_units//ratio)
print(net)

Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_5): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), 

In [6]:
batch_size = 64
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224, root='~/wms/Jupyter/Datasets/FashionMNIST')

lr, epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs)

training on: cuda:7
epoch 1, loss 0.5153, train acc 0.813, test acc 0.865, time 37.0 sec
epoch 2, loss 0.3368, train acc 0.878, test acc 0.889, time 37.1 sec
epoch 3, loss 0.2947, train acc 0.894, test acc 0.893, time 36.8 sec
epoch 4, loss 0.2655, train acc 0.904, test acc 0.902, time 37.2 sec
epoch 5, loss 0.2432, train acc 0.912, test acc 0.907, time 36.4 sec


+ VGG-11通过5个可以重复使用的卷积块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。